# Compatibility between Python and Matlab

Trying to make the g-thermo model compatible between different versions of COBRA ([cobrapy](https://github.com/opencobra/cobrapy) and [cobratoolbox](https://github.com/opencobra/cobratoolbox)).

Benjamín J. Sánchez, 2020-04-07

## 1. Order of annotations

The first noteworthy difference between the two different standards is that cobrapy will always respect the order of annotations in each reaction and stored them like that, whereas cobratoolbox will pre-sort them. This is something that could be improved in the `write_sbml_model` function of cobrapy (as the order doesn't matter anyways), so we will fix that in [this branch](https://github.com/BenjaSanchez/cobrapy/tree/fix/sort-annotations) of my cobrapy's fork. Now we can just read/write the model to confirm that everything is from now on sorted:

In [1]:
# Before:
import cobra
model = cobra.io.read_sbml_model("../model/g-thermo.xml")
for annotation in model.reactions[0].annotation:
    print(annotation)
cobra.io.write_sbml_model(model,"../model/g-thermo.xml")

sbo
kegg.reaction
metanetx.reaction
rhea
ec-code


In [2]:
# After:
model = cobra.io.read_sbml_model("../model/g-thermo.xml")
for annotation in model.reactions[0].annotation:
    print(annotation)

sbo
ec-code
kegg.reaction
metanetx.reaction
rhea


Without counting SBO (which is a separate SBML object), we see that all annotations are now alphabetically ordered.

## 2. Redundant reaction fields

Let's take a look at the reaction notes:

In [3]:
model = cobra.io.read_sbml_model("../model/g-thermo.xml")
print(model.reactions[0].notes)

{'SUBSYSTEM': 'X', 'GENE_ASSOCIATION': '( RTMO00131 or RTMO05104 )', 'KEGG ID': 'R00004', 'ENZYME': '3.6.1.1', 'NAME': 'diphosphate phosphohydrolase; pyrophosphate phosphohydrolase', 'DEFINITION': 'Diphosphate + H2O ⇌ 2 Orthophosphate'}


Both the `SUBSYSTEM` and `GENE_ASSOCIATION` fields are already included in the model (as `groups` and `reaction.gene_reaction_rule` fields, respectively), so we can remove them (as cobratoolbox removes them by default):

In [4]:
for reaction in model.reactions:
    reaction.notes.pop("SUBSYSTEM", None)
    reaction.notes.pop("GENE_ASSOCIATION", None)
cobra.io.write_sbml_model(model,"../model/g-thermo.xml")